In [1]:
import scantelligent.functions as st
from time import sleep
from pynput import keyboard
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from datetime import datetime

Start a session

In [ ]:
session_path, images_path, logfile_path, parameters = st.initialize()

Manual log entries

In [ ]:
message = "This is a manual entry. You can use this to add timestamped comments to the active logfile."
st.logprint(message, logfile = logfile_path)

Simple auto approach

In [ ]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, land_tip = True, timeout = 1000, z_voltage = 220, adjust_percentile = 60)

Simple coarse move in any direction

In [ ]:
st.move_over(direction = "north", steps = 50, z_steps = 10, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 220, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "down")

In [ ]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, land_tip = True, timeout = 1000, z_voltage = 220, adjust_percentile = 60)
st.scan_control(action = "start", scan_direction = "down")

In [22]:
st.coarse_move(direction = "lift", steps = 200)

21:39:44  Coarse motion: 200 steps in the Z+ direction.


Some scanning, coarse moving, changing parameters

In [ ]:
st.scan_control(action = "start", scan_direction = "up")
st.move_over(direction = "north", steps = 20, z_steps = 10, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 220, velocity_threshold = .02, adjust_percentile = 60)
st.change_bias(-2)
st.change_scan_window(center = [30, -100], size = 120, grid = 260, angle = 22)
st.scan_control(action = "start", scan_direction = "down")

In [ ]:
st.scan_control(action = "start", scan_direction = "down", velocity_threshold = .2)

In [ ]:
st.scan_control(action = "stop")

Change functions. They return the old parameters as tuples. Any parameters that are left None or are omitted will remain unchanged.

In [ ]:
V_old = st.change_bias(V = None)
I_old, p_gain_old, t_const_old = st.change_feedback(I = None, p_gain = None, t_const = None)
center_old, size_old, grid_old, angle_old = st.change_scan_window(size = None, grid = None, angle = None)

Tip tracker. Make it time out at 20 seconds. Then return and visualize the x(t), y(t) and z(t) trajectories, with one data point generated every 0.5 seconds. Meanwhile, calculate the surface roughness every 2 seconds.

In [ ]:
txyz = st.tip_tracker(sampling_time = .5, exit_when_still = False, timeout = 20, tracking_info = False, monitor_roughness = True, measurement_interval = 2)
plt.plot(txyz[1], txyz[2])
#plt.plot(txyz[0], txyz[2])
#plt.plot(txyz[0], txyz[3])
plt.show()

Image analysis. Load all files in the current working directory of Nanonis. Change the path argument in get_file_data to get sxm files from another folder

In [3]:
session_path = st.get_session_path()
file_data = st.get_file_data(session_path = session_path)
sxm_files = file_data.image_files
sxm_files_in_folder = [file.split("\\")[-1] for file in sxm_files] # This removes the directory from the path

print("The session path is: " + session_path)
print("SXM files found in this directory: " + "; ".join(sxm_files_in_folder))

The session path is: C:\Data\Qile\082025
SXM files found in this directory: img_0001.sxm; img_0002.sxm; img_0003.sxm; img_0004.sxm; img_0005.sxm; img_0006.sxm; img_0007.sxm; img_0008.sxm; img_0009.sxm; img_0010.sxm; img_0011.sxm; img_0012.sxm; img_0013.sxm; img_0014.sxm; img_0015.sxm; img_0016.sxm; img_0017.sxm; img_0018.sxm; img_0019.sxm; img_0020.sxm; img_0021.sxm; img_0022.sxm; img_0023.sxm; img_0024.sxm; img_0025.sxm; img_0026.sxm; img_0027.sxm; img_0028.sxm; img_0029.sxm; img_0030.sxm; img_0031.sxm; img_0032.sxm; img_0033.sxm; img_0034.sxm; img_0035.sxm; img_0036.sxm; img_0037.sxm; img_0038.sxm; img_0039.sxm; img_0040.sxm; img_0041.sxm; img_0042.sxm; img_0043.sxm; img_0044.sxm; img_0045.sxm; img_0046.sxm; img_0047.sxm; img_0048.sxm; img_0049.sxm; img_0050.sxm; img_0051.sxm; img_0052.sxm; img_0053.sxm; img_0054.sxm; img_0055.sxm; img_0056.sxm; img_0057.sxm; img_0058.sxm; img_0059.sxm; img_0060.sxm; img_0061.sxm; img_0062.sxm; img_0063.sxm; img_0064.sxm; img_0065.sxm; img_0066.sxm; 

Get the data for a scan. First make sure the scan file exists and contains the target channel. Then, check how clipping and background subtraction affects the scan image.

In [ ]:
file_no = 10
target_channel = "Z"
dirxn = 0 # dirxn = 0 means forward; dirxn = 1 means backward
clip_percentiles = [20, 80]

if file_no > len(sxm_files) - 1: print("Error: This file does not exist.")
else:
    scan_data = st.get_scan(sxm_files[file_no]) # Import all the scan data using Nanonispy
    V = scan_data.V
    size = scan_data.header.get("scan_range", [100, 100])
    
    channels = scan_data.channels
    find_channel = np.where(np.array(channels) == target_channel)[0]
    
    if len(find_channel) != 1: print("Error: The target channel was not found")
    else:
        channel_index = find_channel[0]
        filename = images_path + "\\" + channels[channel_index] + f"_{file_data.image_indices[file_no]:03d}.png"
        img_raw = scan_data.scans[channel_index, dirxn]
        
        clip_values = st.clip_range(img_raw, method = "percentiles", values = clip_percentiles)
        histogram = st.get_image_statistics(img_raw).histogram
        img_b = st.background_subtract(img_raw)
        clip_values_b = st.clip_range(img_b, method = "percentiles", values = clip_percentiles)
        histogram_b = st.get_image_statistics(img_b).histogram

        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(img_raw, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")
        ax[1].imshow(img_b, vmin = clip_values_b[0], vmax = clip_values_b[1], cmap = "gray")
        fig.set_size_inches(10, 5)
        plt.show()

        fig, ax = plt.subplots()
        ax.plot(-histogram[1], histogram[0] - np.mean(histogram[0]))
        ax.plot(histogram_b[1], histogram_b[0])
        print("Image histograms:")
        plt.show()
        
    print(filename)

Check the background subtraction and clipping

In [ ]:
background = True

clip_values = st.clip_range(img_raw, method = "percentiles", values = clip_percentiles)
histogram = st.get_image_statistics(img_raw).histogram
img_b = st.background_subtract(img_raw)
clip_values_b = st.clip_range(img_b, method = "percentiles", values = clip_percentiles)
histogram_b = st.get_image_statistics(img_b).histogram

if background:
    img = img_b
    clip_values = clip_values_b
else: img = img_raw

fig, ax = plt.subplots()
ax.imshow(img, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")
fig.set_size_inches(10, 5)
plt.show()
plt.imsave(filename, img, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")

print("Successfully saved " + filename)